In [2]:
import numpy as np
from math import log
import operator

def count_entropy(dataset):
    total_count = len(dataset)
    map_labels = {}
    labels = [x[-1] for x in dataset]
    for label in labels:
        map_labels.setdefault(label,0)
        map_labels[label] += 1
  
    entropy = 0.0
    for value in map_labels.values():
        prob = value * 1.0 / total_count
        entropy += - prob  * log(prob , 2)
    return entropy

def split_dataset(dataset, feature, value):
    sub_dataset = []
    for item in dataset:
        if item[feature] == value:
            newitem = item[:feature]
            newitem.extend(item[feature+1:])
            sub_dataset.append(newitem)
    return sub_dataset

def getbest_feature(dataset):
    base_entropy = count_entropy(dataset)
    
    best_feature = -1
    best_infogain = 0.0
    
    for i in range(len(dataset[0])-1):
        feature_values = set([item[i] for item in dataset])
        infogain = 0.0
        inforatio = 0.0
        for k in feature_values:
            subdataset = split_dataset(dataset, i, k)
            prob = float(len(subdataset)) / len(dataset) 
            infogain +=  prob  * count_entropy(subdataset)
            inforatio += - prob  * log(prob , 2)    
        
        if inforatio == 0:
            continue
        
        infogain = (base_entropy - infogain) / inforatio
        if inforatio > best_infogain:
            best_feature = i
            best_infogain = inforatio
    
    #print 'best_fature',best_feature
    return best_feature
        
    
def create_decisiontree(dataset, label):
    #print 'dataset',dataset
    #print 'lable',label
    
    #所有数据一个类别
    labels = [instance[-1] for instance in dataset]
    if labels.count(labels[0]) == len(labels):
        return labels[0]

    #no data,only labels,当切分的不能再切分的时候
    if len(dataset[0]) == 1:
        map_labels = {}
        labels = [x[-1] for x in dataset]
        for res in labels:
            map_labels.set_default(res,0)
            map_labels[res] += 1
        sorted(map_labels.iteritems(), key=operator.itemgetter(1), reverse=True)
        print 'res',map_labels
        return map_labels[0][0]
    
    #count base entropy
    i = getbest_feature(dataset)
    
    feature_name = label[i]
    del label[i]
    
    feature_values = set([item[i] for item in dataset])
    
    decision_tree = {feature_name:{}}
    for val in feature_values:
        sub_dataset = split_dataset(dataset, i, val)
        decision_tree[feature_name][val] = create_decisiontree(sub_dataset, label)
    
    return decision_tree

def predict(test_data, labels, tree):

    root_feature = list(tree.keys())[0]
    
    feature_idx = labels.index(root_feature)
    subtree = tree[root_feature]
    label = ''
    for item in subtree:
        if item == test_data[feature_idx]:
            if type(subtree[item]).__name__ == 'dict':
                label = predict(test_data, labels, subtree[item])
            else:
                label = subtree[item]
            break
    return label
    
        

if '__main__' == __name__:
    
    train_set = [[0, 0, 0, 0, 'N'], 
               [0, 0, 0, 1, 'N'], 
               [1, 0, 0, 0, 'Y'], 
               [2, 1, 0, 0, 'Y'], 
               [2, 2, 1, 0, 'Y'], 
               [2, 2, 1, 1, 'N'], 
               [1, 2, 1, 1, 'Y']]
    labels = ['outlook', 'temperature', 'humidity', 'windy']
    
    
    
    decision_tree = create_decisiontree(train_set, labels)
    
    print  'decision tree',decision_tree
    test_dataset = [2, 2, 0, 1]
    labels = ['outlook', 'temperature', 'humidity', 'windy']
    print predict(test_dataset, labels, decision_tree)

decision tree {'outlook': {0: 'N', 1: 'Y', 2: {'temperature': {1: 'Y', 2: {'windy': {0: 'Y', 1: 'N'}}}}}}
N
